In [1]:
pip install torch transformers langchain gradio sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
import gradio as gr

In [3]:
from transformers import BioGptForCausalLM, BioGptTokenizer,pipeline
import torch
from peft import PeftModel

base_model = BioGptForCausalLM.from_pretrained(
    "microsoft/biogpt",  # Original base model
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, "/kaggle/input/notebook36cd12f0a1/biogpt-qlora-medical")
model = model.merge_and_unload() 


2025-05-17 10:37:57.290619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747478277.502838      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747478277.563929      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

In [4]:
from datasets import load_from_disk

val_dataset = load_from_disk("/kaggle/input/notebook36cd12f0a1/val_dataset")

In [5]:
print(len(val_dataset))

20000


In [6]:
from datasets import Dataset
def format_example(example):
    return {
        "text": f"### Patient: {example['Patient']}\n### Doctor:"
    }
val_dataset = Dataset.from_dict(val_dataset[:])
val_dataset = val_dataset.map(format_example)
print(val_dataset[0])

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

{'Description': 'What causes swelling in the cheek where the root canal was done?', 'Patient': 'Monday I had a root canal done by an orthodontist. He prescribed me 800 mg Motrin and hydrocodone. I took the Motrin as prescribed. The side of my cheek where the RC was done began swelling Wednesday night. He called in penicillin go 500mg and I began taking it it is Friday night. Should he have prescribed a stronger antibiotic.? I dont have anymore pain  but should I still take the Motrin? If I go to the emergency room would they be able to help? Will taking the antibiotic reduce the swelling alone', 'Doctor': 'Thanks for your query, I have gone through your query.The swelling could be because of the endodontic flare up pr it can be because of the remnant infection. Consult a oral physician and get a radiograph to rule out whether all the canals have been filled properly and to rule out any infection.If there is defect in RCT and infection you have to get it retreated. If it is because of t

In [7]:
!pip install -q evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [8]:
import evaluate
from tqdm import tqdm
import numpy as np

max_model_length = 1024
max_new_tokens = 128
max_input_length = max_model_length - max_new_tokens  # 896

predictions = []
references = []
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
for example in tqdm(val_dataset.select(range(2000))):
    input_text = example["text"]
    reference_text = example["Doctor"]

    # Tokenize input with truncation to max_input_length
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_input_length,
    ).to(model.device)

    # Generate prediction
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            repetition_penalty=1.2,
        )

    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = pred_text.split("Doctor:")[-1].strip()
    predictions.append(response)
    references.append(reference_text)



vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

100%|██████████| 2000/2000 [1:21:12<00:00,  2.44s/it]


In [9]:
# Compute metrics
# For BLEU and METEOR, refs need to be list of lists
rouge_metric = evaluate.load("rouge")   
bleu_metric = evaluate.load("bleu") 
meteor_metric = evaluate.load("meteor") 
bleu_metric.add_batch(
    predictions=predictions,   
    references=[[ref] for ref in references]  
)
meteor_metric.add_batch(predictions=predictions, references=references)
rouge_metric.add_batch(predictions=predictions, references=references)

# Show results
rouge_results = rouge_metric.compute()
bleu_result = bleu_metric.compute()
meteor_result = meteor_metric.compute()

print("ROUGE Scores:")
for k, v in rouge_results.items():
    print(f"{k}: {v:.4f}")

print(f"\nBLEU Score: {bleu_result['bleu']:.4f}")
print(f"METEOR Score: {meteor_result['meteor']:.4f}")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


ROUGE Scores:
rouge1: 0.2372
rouge2: 0.0404
rougeL: 0.1373
rougeLsum: 0.1372

BLEU Score: 0.0322
METEOR Score: 0.1903
